# Data Wrangling

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import requests
import time
import random

## Gather

### Scrape BMO Mutual Funds List

FYI, here is the the link to the website: [BMO Mutual Funds List](https://www.bmo.com/home/personal/banking/investments/mutual-funds/navigator/funds/mutual-funds-list/funds-overview).

The website lists all BMO mutual funds in the market. Since it is in a tabular format (`tr` and `td` tags), it's easy to parse the HTML and extract the data. Here is a quick summary of the process:

1. Create a Chrome driver using `Selenium`, and go to the specified URL.
2. Parse the HTML, and find all `tr` tags with the `valign` attribute set to `center`.
3. Extract data from each `tr` into a list. 
4. Convert the list into a Pandas DataFrame.

In [ ]:
def extractData():
    """Extracts the data from the BMO website.
    
    Returns:
        A list of dictionary
    """

    # create a list to store cleaned data
    results = list()
    
    # wait until the tr appear
    trs = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//tr[@valign="center"]'))
    )
    
    for tr in trs:
    
        # the BMO fund ID
        fundID = tr.get_attribute('id')
        fundID = fundID.rsplit("_")[-1]

        # locate the <a> and extract href
        profileURL = tr.find_element_by_tag_name('a').get_attribute('href')    

        # the attribtues are stored in the listCellWithBorders class
        # find them all, and extract text

        attrs = tr.find_elements_by_class_name('listCellWithBorders')
        fundName, _, price, assetClass, assets, dateCreated = list(map(lambda x: x.text, attrs))

        # insert the data into the list
        results.append({
            'Fund ID': fundID,
            'Fund Name': fundName,
            'Price': price,
            'Asset Class': assetClass,
            'Assets': assets,
            'Date Started': dateCreated,
            'Fund Profile': profileURL
        })
        
    return results

In [ ]:
url = 'https://www.bmo.com/home/personal/banking/investments/mutual-funds/navigator/funds/mutual-funds-list/funds-overview'

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.get(url)

In [ ]:
bmo = extractData()

###  Scrape BMO portfolios 

The portfolio data is under the drop-down at the top right. The code below does the following actions:

1. Locate the drop-down.
2. Go to each option in the drop-down, and extract the data.
3. Append the data to the original list

In [ ]:
# skip the first two options
page_index = 3 

In [ ]:
while page_index < 10:
    
    # locate the drop-down     
    select = Select(driver.find_element_by_id('portfolio'))
    select.select_by_index(page_index)
    
    # increment the page counter     
    page_index += 1
    
    # append new data to the original list
    bmo += extractData()

In [ ]:
driver.quit()

In [ ]:
# conver the data into a pandas dataframe
bmo = pd.DataFrame(bmo)

### Extract Historical Price

Now, we already have the mutual funds list. However, the price column represents the latest price. It would be great if we can collect all historical data. After inspecting the following URL: `https://bmomf.lipperweb.com/Profile/HistoryView?symbol=45121:17695&tab=History&timeFrame=Year&interval=0&period=1&startDate=&endDate=&isTimePeriodChange=true&lang=en`, I figured out the pattern to retrieve the recorded information. The URL consists of the following components:

1. The base URL: `https://bmomf.lipperweb.com/Profile/HistoryView`
2. The query strings:
    
    * symbol: 45121:17695
    * etc.

Noted that 17695 is the Fund ID, and the code 45121 never changes. If we request this URL by iterating all our fund IDs, we can get all historical prices. Here is the code:

In [ ]:
# specify the parameters and headers for the HTTP get request
params = {
    "symbol": "",
    "tab": "History",
    "timeFrame": "Year",
    "interval": "0",
    "period": "1",
    "startDate": "",
    "endDate": "",
    "isTimePeriodChange": True,
    "lang": "en",
}

headers = {
    "Accept": "text/html, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "_ga=GA1.2.1526468817.1615351851; _gid=GA1.2.881144454.1615351851; _gat=1",
    "Host": "bmomf.lipperweb.com",
    "Referer": "https://bmomf.lipperweb.com/bmomf/profile/?symbol=45121:94792&lang=en",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
}

In [ ]:
baseURL = 'https://bmomf.lipperweb.com/Profile/HistoryView'

In [ ]:
results = list()

In [ ]:
for index, fund in bmo.iterrows():
    
    print(fund['Fund Name'])
    
    # use fstring to replace the value
    params['symbol'] = f"45121:{fund['Fund ID']}"

    # send the request
    res = requests.get(baseURL, params=params, headers=headers)
    
    time.sleep(random.uniform(0.0, 2.0))
    
    # the response is in HTML format, I use BeautifulSoup to parse it
    document = BeautifulSoup(res.content, 'html.parser')
    
    # the data is stored in the <td></td>. find them all
    tds = document.find_all('td', {'style': 'text-align: center;'})
    
    # extract the inner text for all tds
    values = [ td.text.strip() for td in tds ]
    
    # We have a 1d array, but each data point consists of 6 columns. 
    # The following code turns every six elements into a list. In 
    # other words, we are converting the 1d array to a 2d array
    # Here is an quick example. Before:
    # ['8/25/2021', 13.8073, ..., '8/24/2021', 13.8541, ...]
    # After: [['8/25/2021', 13.8073, ...], ['8/24/2021', 13.8541, ...]]
    
    values = [ [fund['Fund ID']] + values[i:i+6] for i in range(0, len(values), 6)]
    
    # append the values to the retuslt list
    results += values

In [ ]:
# the column name
cols = [
    'Fund ID',
    'Effective Date',
    'NAV',
    'Income',
    'Capital Gain',
    'Total Distribution',
    'Reinvestment Price'
]

# convert the data into a df
prices = pd.DataFrame(results, columns=cols)

## Assess

### Inspect `bmo` & `prices`

#### First glance

**Issue**: the `Price` and the `Fund Profile` columns in `bmo` is useless.

In [ ]:
bmo.head(2)

In [ ]:
prices.head(2)

#### Check Data Types & Missing Values

**Issue**: Noted that all column types are strings.

In [ ]:
display(bmo.info())
display(prices.info())

**Issue**: missing values shown as empty string: `''`

In [ ]:
bmo[bmo['Fund Name'] == ''].head(3)

**Issue**: missing value shown as `-`

In [ ]:
prices.loc[1:3, 'Income':'Reinvestment Price']

#### Check Duplicates

In [ ]:
bmo.duplicated().sum(), prices.duplicated().sum() 

### Summary of the Inspections

#### Quality - Content Issue

* Data type issue: all column types are strings.
* Missing values: 
    
    1. Some funds do not have a Fund Name.
    2. Missing values display as `''` and `-`
    
#### Tidiness - structurnal issue

* the `Price` and the `Fund Profile` columns in `bmo` is useless.

## Clean

### Drop the `Price` and the `Fund Profile` columns

#### Define

We will drop the columns since we already have a DataFrame for `Price`, and the URL doesn't help with the analysis.

#### Code

In [ ]:
bmo.drop(columns=['Price', 'Fund Profile'], inplace=True)

#### Test

In [ ]:
bmo.columns

### Drop rows with empty string: `''`

#### Define

BMO did not provide the name for some funds. Since we can't and it is inappropriate to make up the fund name. We will drop the rows that have a fund name as an empty string.

#### Code

In [ ]:
bmo = bmo[bmo['Fund Name'] != '']

#### Test

In [ ]:
bmo['Fund Name'].duplicated().sum()

### Fill `-` with Nan

#### Define

BMO puts `-` to a cell when there is a missing value. Since the official value for missing values is `nan`, we will replace `-` with `nan`.

#### Code

In [ ]:
prices = prices.applymap(lambda x: np.nan if x == '-' else x)

#### Test

In [ ]:
prices.applymap(lambda x: x == '-').sum()

### Convert column data types

#### Define

* The `bmo` DataFrame:

    * convert `Assets` to `float`.
    * convert `Date Started` to `datetime`.

* The `prices` DataFrame:

    * convert `Effective Date` to `datetime`.
    * convert all columns except `Fund ID` and `Effective Date` to `float`.


#### Code

##### `bmo`

Noted that the data has comma separators

In [ ]:
bmo.loc[bmo['Assets'].apply(lambda x: ',' in x), 'Assets'].head(5)

In [ ]:
# create a lambda function to clean numeric data
cleanNum = lambda x: float(x.lstrip('$').replace(',', '')) if type(x) == str else x

In [ ]:
bmo.loc[:, 'Assets'] = bmo.Assets.apply(cleanNum)

In [ ]:
bmo.loc[:, 'Date Started'] = pd.to_datetime(bmo['Date Started'])

##### `prices`

In [ ]:
prices.loc[:, 'Effective Date'] = pd.to_datetime(prices['Effective Date'])

In [ ]:
targetCols = prices.columns[2:]

In [ ]:
for col in targetCols:
    prices.loc[:, col] = prices[col].apply(cleanNum)

#### Test

In [ ]:
bmo.dtypes, prices.dtypes

## Export the datasets

In [ ]:
bmo.to_csv('resources/BMOMutualFunds.csv', index=False, encoding='utf-8')

In [ ]:
prices.to_csv('resources/BMOFundPrices.csv', index=False, encoding='utf-8')